# AdviescollegeScraper

Doel: 
- Informatie verzamelen van de verschillende Nederlandse adviescolleges.

Toepassing: Bachelor-scriptie Informatiekunde, Universiteit van Amsterdam

Contact: mitchell.malaihollo@student.uva.nl

In [1]:
# imports
import csv
import re
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

In [2]:
# Definieert bezoekbare kern URL's
coreURL = "https://organisaties.overheid.nl"
subCoreURL = "https://organisaties.overheid.nl/Adviescolleges"

# Creëert een identificeerbare browser-achtige user-agent 
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) "
        "Gecko/20100101 Firefox/124.0 "
        "AdviescollegeScraper (+mailto:14605120@student.uva.nl; "
        "voor BSc-scriptie Informatiekunde UvA; "
        "doel: informatie inwinnen over verschillende adviescolleges)"
    )
}

In [3]:
def retrieve(url: str) -> BeautifulSoup:
    """Haalt HTML-gegevens op van de opgegeven URL en retourneert een BeautifulSoup-object"""
    response = requests.get(url, headers=HEADERS, timeout=10)
    response.raise_for_status()

    return BeautifulSoup(response.text, "html.parser")

In [4]:
def retrieveLinks() -> list[str]:
    """"Haalt de verschillende unieke adviescolleges op uit de overzichtspagina"""
    data = retrieve(subCoreURL)
    links = set()

    for x in data.find_all("a", href=True):
        url = urljoin(subCoreURL, x['href'])

        # filtert alleen links die een coreURL bevatten en cijfers bevatten (unieke identificatie voor adviescolleges)
        if coreURL in url and re.search(r"/\d+/", url):
            links.add(url)

    return links

In [5]:
def parse_detail(url: str) -> dict:
    """Parseer gegevens van de gegeven url"""
    soup = retrieve(url)
    text = soup.get_text(" ", strip=True)

    # Scope
    scope = ""
    scope = re.search(r"\b(Permanent|Tijdelijk|Eenmalig)\s+adviescollege\b", text, re.IGNORECASE)
    scope = scope.group(1).capitalize()

    # Kaderwet
    kaderwet = ""
    table = soup.select_one('table[data-roo-element="element-kaderwet-adv"]')
    if table:
        cell = table.select_one("td span")
        if cell:
            kaderwet = cell.get_text(strip=True)
    kaderwet = kaderwet.replace("(zieKaderwet adviescolleges)", "").strip()
    
    # Naam
    naam = soup.select_one('h1[data-roo-element="element-naam-en-afkorting"]')
    direct_text = "".join(naam.find_all(string=True, recursive=False)).strip() if naam else ""
    naam = re.sub(r"\s+", " ", direct_text).strip()
    if '()' in naam:
            naam = naam.replace("()", "").strip()

    # Afkorting
    abbr_el = soup.select_one("abbr")
    afkorting = abbr_el.get_text(strip=True) if abbr_el else ""

    # Adviescollege URL
    adviescollegeURL_elem = soup.select_one('section#contactgegevens td[data-before="Internet"] a[href]')
    adviescollegeURL = adviescollegeURL_elem["href"].strip() if adviescollegeURL_elem else None
    adviescollegeURL = adviescollegeURL or None
        
    # Identification TOOIURI Code
    TOOIURI = soup.select_one('section#identificatiecode td[data-before="tooiUri"] a[href]')
    TOOIURI = TOOIURI["href"].strip() if TOOIURI else ""

    # Actief vanaf
    actiefVanaf = soup.select_one('span[data-roo-element="element-active-from-till"]')
    actiefVanaf = actiefVanaf.get_text(strip=True) if actiefVanaf else ""
    actiefVanaf = re.search(r"\d{2}-\d{2}-\d{4}", actiefVanaf)
    if actiefVanaf:
        actiefVanaf = actiefVanaf.group()
        
    # Classificaties
    classificaties = []
    header = soup.find(lambda tag: tag.name in ["h2", "h3"] and "Classificaties" in tag.get_text())
    if header:
        table = header.find_next("table")
        if table:
            for row in table.find_all("tr"):
                cells = row.find_all("td")
                if len(cells) >= 2:
                    raw = cells[1].get_text(" ", strip=True)

                    # neem alleen het eerste woord (voor de punt of spatie)
                    short = raw.split(".", 1)[0]     # knip bij eerste punt
                    short = short.split(" ", 1)[0]  # knip bij eerste spatie
                    short = short.strip().lower()   # consistent maken

                    if short and short not in ["classificatie"]:
                        classificaties.append(short)

    classificaties_str = ", ".join(classificaties)

    return {
    "Naam": naam, 
    "Afkorting": afkorting, 
    "Adviescollege Scope": scope, 
    "Actief vanaf": actiefVanaf,
    "Kaderwet van toepassing": kaderwet, 
    "Classificaties": classificaties_str,
    "Register van overheidsorganisaties URL": url, 
    "Adviescollege URL": adviescollegeURL, 
    "TOOI-URI": TOOIURI
    }

In [6]:
def main():
    exportTarget = "AdviescollegeScraperData.csv"

    links = retrieveLinks()
    numLinks = len(links)
    print("Op zoek naar adviesorganen in het Register van Overheidsorganisaties.")
    print("Totaal aantal gevonden adviescolleges:" + str(numLinks))
    print(" ")

    fieldnames = [
        "Naam",
        "Afkorting",
        "Adviescollege Scope",
        "Actief vanaf",
        "Kaderwet van toepassing",
        "Classificaties",
        "Register van overheidsorganisaties URL",
        "Adviescollege URL",
        "TOOI-URI",
    ]

    with open(exportTarget, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter=";")
        writer.writeheader()
        count = 0
        totalCount = 0
        
        for url in links:
            data = parse_detail(url)
            if data is False:
                totalCount += 1
                print(totalCount)
                continue
            else:
                writer.writerow(data)
                count += 1
                totalCount += 1
                print("[newCount = " + str(count) + "/" + str(totalCount) + "/" + str(numLinks) + "]: " + str(data['Naam']))
    
    print(" ")
    print(f"Klaar. " + str(count) + " adviescolleges opgeslagen.")

if __name__ == "__main__":
    main()

Op zoek naar adviesorganen in het Register van Overheidsorganisaties.
Totaal aantal gevonden adviescolleges:106
 
[newCount = 1/1/106]: Comité van toezicht EFMB
[newCount = 2/2/106]: Staatscommissie tegen Discriminatie en Racisme
[newCount = 3/3/106]: Kiesraad
[newCount = 4/4/106]: Adviescollege Rechtspositie Politieke Ambtsdragers
[newCount = 5/5/106]: Wetenschappelijke Adviescommissie Convenant Onbedwelmd Ritueel Slachten
[newCount = 6/6/106]: Adviescollege ICT-toetsing
[newCount = 7/7/106]: Adviescommissie beoordeling aangeboden cultuurbezit uit nalatenschappen
[newCount = 8/8/106]: Adviescommissie seed capital technostarters
[newCount = 9/9/106]: Commissie onderzoek naar Binnenlandse Afstand en Adoptie
[newCount = 10/10/106]: Adviescommissie Kredietcommissie
[newCount = 11/11/106]: College financieel toezicht Bonaire, Sint Eustatius en Saba
[newCount = 12/12/106]: Adviescommissie vroegefasefinanciering MKB-ondernemers en innovatieve starters
[newCount = 13/13/106]: Commissie van to